# Test Neuroevolución

In [79]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
import pandas as pd
from sklearn.preprocessing import StandardScaler  

In [80]:
from deap import algorithms
from deap import base
from deap import creator
from deap import tools
import random
import pickle

Importación del scaler
http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

In [81]:
scaler = StandardScaler() 

#### Cargo los datos y retiro el Timestamp y el score

In [82]:
df = pd.read_csv("gameStates.csv", sep=',', header=None)
df.columns = ['timeStamp','Px', 'Py', 'heat', 'Exp1','Eyp1','Exp2','Eyp2', 'Ex1', 'Ey1', 'Eh1', 'Ex2', 'Ey2', 'Eh2', 'Ex3', 'Ey3', 'Eh3', 'Ex4', 'Ey4','Eh4','Ex5', 'Ey5','Eh5','Ex6', 'Ey6','Eh6','ray1','ray2','ray3','ray4','ray5','ray6','ray7','ray8','ray9','ray10','ray11','ray12','ray13','ray14','ray15','ray16','ray17','ray18','ray19','ray20','ray21','ray22','ray23','ray24','ray25','ray26','ray27','score',"VKey","HKey","Shooting"]
df.drop(['timeStamp','score'],axis=1,inplace=True)
df = df[1:5001]


In [83]:
#Transforma el porblema multioputput en multilabel
def multioutput2multilabel(row):
    values = pd.Series([(row["VKey"] == "UpArrow"),(row["VKey"] == "DownArrow"),(row["HKey"] == "LeftArrow"),(row["HKey"] == "RightArrow"),row["Shooting"]])
    return pd.concat([row[:len(row)-3], values.astype(float)])
df = df.apply(multioutput2multilabel,axis=1)

In [84]:
#Defino los elementos train_data y target_data
train = df.drop([0, 1, 2],axis=1).values
labels = df[[0,1,2]].values

In [85]:
#Preprocesado obligatorio en este tipo de problemas -Leer más sobre ello-
scaler.fit(train)
train = scaler.transform(train)

In [86]:
#Se crea el clasificador, se hace el entrenamiento y se guarda.
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(15,), random_state=1)
trained_data = clf.fit(train, labels)
pickle.dump(trained_data, open("clf.sav", 'wb'))

In [87]:
[coef.shape for coef in clf.coefs_]

[(54, 15), (15, 3)]

In [88]:
from copy import deepcopy
model = deepcopy(clf)

In [89]:
#Transforma de en  a coefs y de coefs a gen
def gen2Coefs(gen,sizes,shapes):
    coefs = []
    splits = np.split(gen, [sizes[0]])
    for i in range(len(splits)):
        coefs.append(splits[i].reshape(shapes[i]))
    return coefs

def coefs2gen(coefs,sizes,shapes):
    return np.concatenate((coefs[0].flatten(),coefs[1].flatten()))

print(len(coefs2gen(clf.coefs_,sizes,shapes)))

855


In [90]:
#Función de evaluación
from sklearn.metrics import accuracy_score
def MLPFitness(individual):
    coefs=gen2Coefs(individual,sizes,shapes)
    model.coefs_ = coefs
    preds = model.predict(Xprima)
    acc = accuracy_score(y, preds)
    return acc,


In [92]:
shapes = [coef.shape for coef in clf.coefs_] # solo voy a ajustar los pesos
sizes =[coef.size for coef in clf.coefs_]

In [93]:
Xprima = train
y = labels
MLPFitness(coefs2gen(clf.coefs_,sizes,shapes))

(0.54079999999999995,)

## Definicion del algoritmo

In [95]:
#Función de initPopulation
def initPopulation(pop, ind_random, ind_guess,n_guess=1,n=10):
    pop = []
    n_random = n - n_guess
    for i in range(n_random):
        pop.append(ind_random())
    for i in range(n_guess):
        pop.append(ind_guess())
    return pop

def getGuest():
    return self.load_model("Forest.sav")

def evalFunct():
    

toolbox = base.Toolbox()
toolbox.register("attr_int", random.uniform, -10, 10)
toolbox.register("individual_rnd", tools.initRepeat, creator.Individual, toolbox.attr_int, n=855)
toolbox.register("individual_guess", creator.Individual(getGuest))
toolbox.register("population_mix",initPopulation,list,toolbox.individual_rnd, toolbox.individual_guess)       


#print(toolbox.population_mix(n=10, n_guess=0))
genes = toolbox.population_mix(n=10, n_guess=1)
toolbox.register("evaluate", evalFunct)




[Individual([ 4.9070152 , -4.95268587,  4.02918582,  8.77409947, -4.70337112,
        1.5187256 , -7.67521774,  5.15061826, -4.6106123 , -4.61368033,
        2.55357772,  5.33171871, -4.49273792, -9.90701764, -5.4953862 ,
        3.06736601,  5.88145735,  1.31638114, -0.55286305, -1.67054305,
        3.30672968,  2.22836211, -3.34671984, -4.73106985,  1.92701934,
        9.91686512,  4.25974277,  4.90269078,  6.25976051, -6.23387729,
       -8.84963899,  4.64631465,  8.00921794, -2.96750641, -7.161398  ,
       -6.62100368, -1.41434921, -5.27229953,  9.84406763,  2.49966472,
       -7.77038306,  1.8137862 ,  9.79785783, -8.31193725, -5.13991408,
        7.73925591, -9.16490644,  5.80884741,  8.49845227,  0.22201602,
       -3.45872098, -6.28297263,  7.37699819,  1.66023424,  6.18030473,
        1.14588715, -5.90660029, -6.29567948,  0.61956045, -6.52891439,
       -0.86739672,  5.90874923,  2.63857941, -5.71487178,  6.48207386,
        3.07633203,  2.13823126, -3.66455919, -0.67657295,